<H1> CURSO DE INTRODUÇÃO A MACHINE LEARNING - LDS </H1>
<h2> Preprocessamento de Dados Estruturados </h2>


In [8]:
%matplotlib inline

In [9]:
#IMPORTS 

# Biblioteca Padrão de Matemática
import math

#NumPy
import numpy as np

#scipy stats
from scipy import stats

#matplotlib pyplot
import matplotlib.pyplot as plt

#taamnho das figuras
plt.rcParams['figure.figsize'] = (8,4)

#copias de listas
from copy import deepcopy


<table style="width:100%">
  <tr>
    <th align = "left">Atributo</th>
    <th align = "left">Valores Textuais</th> 
  </tr>
  <tr>
    <td align = "left">Formato</td>
    <td align = "left">Esférico, Oval, Alongado</td> 
  </tr>
  <tr>
    <td align = "left">Fruta Cítrica</td>
    <td align = "left">Sim,Não</td> 
  </tr>
  <tr>
    <td align = "left">Cor</td>
    <td align = "left">Amarelo,Vermelho,Alaranjado,Verde</td> 
  </tr> 
   <tr>
    <td align = "left">Rugosidade</td>
    <td align = "left">Lisa,Rugosa</td> 
  </tr><tr>
    <td align = "left">Cheiro</td>
    <td align = "left">Não,Sim</td> 
  </tr>
</table>

In [10]:
# Dataset 4 - Frutas
# Atributos : Formato, Fruta Cítrica, Cor, Rugosidade, Cheiro
#
dadosBrutos = [['esferico','sim','alaranjado','rugosa','nao','LARANJA'],
               ['esferico','nao','vermelho','lisa','nao','MACA'],
               ['esferico','sim','alaranjado','rugosa','sim','TANGERINA']]

<h2> Codificador de Rótulos (Label Encoder) </h2>

In [11]:
def codificar(valorAtual,valoresPossiveis):
    s = list(set(sorted(valoresPossiveis)))
    return s.index(valorAtual)

In [12]:
codificar('sim',['sim','nao'])

0

In [13]:
codificar('nao',['sim','nao'])

1

In [14]:
codificar('esferico',['esferico','oval','alongado'])

0

In [15]:
codificar('oval',['esferico','oval','alongado'])

2

<h2> Transformando o Dataset <h2>

In [16]:
def criaDominios(ds):
    # trata-se da matriz transposta sem repetição dos valores
    n  = len(ds[0])-1
    m  = len(ds)
    doms = []
    for i in range(n):
        l = []
        for j in range(m):
            if (ds[j][i] not in l) and isinstance(ds[j][i], str): 
                l.append(ds[j][i])
        l.sort()
        if l not in doms:
            doms.append(l)
        doms.sort()
    return doms
    

In [17]:
criaDominios(dadosBrutos)

[['alaranjado', 'vermelho'], ['esferico'], ['lisa', 'rugosa'], ['nao', 'sim']]

In [18]:
def localizalinha(str,dom):
    for i in dom:
        if str in i:
            return i
    return -1

In [19]:
localizalinha('alaranjado',criaDominios(dadosBrutos))

['alaranjado', 'vermelho']

In [20]:
#python puro
def transformacaoDataset(ds):
    n = len(ds)
    m = len(ds[0])-1
    dom = criaDominios(ds)
    for i in range(n):
        for j in range(m):
            linha = localizalinha(ds[i][j],dom)
            ds[i][j] = codificar(ds[i][j],linha)
    return ds

In [21]:
nDs = transformacaoDataset(deepcopy(dadosBrutos))

In [22]:
nDs

[[0, 0, 0, 0, 1, 'LARANJA'],
 [0, 1, 1, 1, 1, 'MACA'],
 [0, 0, 0, 0, 0, 'TANGERINA']]

In [16]:
dadosBrutos

[['esferico', 'sim', 'alaranjado', 'rugosa', 'nao', 'LARANJA'],
 ['esferico', 'nao', 'vermelho', 'lisa', 'nao', 'MACA'],
 ['esferico', 'sim', 'alaranjado', 'rugosa', 'sim', 'TANGERINA']]

In [17]:
def distanciaEuclidiana(a,b): 
    d = 0
    for i,j in zip(a,b):
        d+= (i-j)**2
    return  math.sqrt(d) 

In [18]:
distanciaEuclidiana(nDs[0][:-1],nDs[1][:-1]) #laranja e maca

1.7320508075688772

In [19]:
distanciaEuclidiana(nDs[0][:-1],nDs[2][:-1]) #laranja e tangerina

1.0

In [20]:
distanciaEuclidiana(nDs[1][:-1],nDs[2][:-1]) #maca e tangerina

2.0

<h3> Label Encoder no Scikit Learning <h3>

In [21]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['esferico','oval','alongado'])

LabelEncoder()

In [22]:
list(le.classes_)

['alongado', 'esferico', 'oval']

In [23]:
le.transform(["oval", "oval", "esferico"]) 

array([2, 2, 1], dtype=int64)

In [24]:
list(le.inverse_transform([2, 2, 1]))

['oval', 'oval', 'esferico']

<h3> Qual o problema com o label encoder ? </h3>

In [25]:
def transposta(matriz):
    n = len(matriz)
    m = len(matriz[0])
    novaMatriz = []
    for i in range(m):
        l = []
        for j in range(n):
            l.append(matriz[j][i])
        novaMatriz.append(l)
    return novaMatriz

In [26]:
transposta(dadosBrutos)

[['esferico', 'esferico', 'esferico'],
 ['sim', 'nao', 'sim'],
 ['alaranjado', 'vermelho', 'alaranjado'],
 ['rugosa', 'lisa', 'rugosa'],
 ['nao', 'nao', 'sim'],
 ['LARANJA', 'MACA', 'TANGERINA']]

In [27]:
def novosRotulos(ds,rots):
    dst = transposta(ds)
    nrots =[]
    for i,j in zip(rots,dst[:-1]):
        nr = set(j)
        if "sim" in nr or "nao" in nr:
            nrots.append(i)
        else:
            for k in nr:
                nrots.append(k)
    return nrots

In [28]:
rotulos = 'Formato','Fruta Cítrica','Cor', 'Rugosidade', 'Cheiro'
novosRotulos(dadosBrutos,rotulos)

['esferico',
 'Fruta Cítrica',
 'vermelho',
 'alaranjado',
 'lisa',
 'rugosa',
 'Cheiro']

In [29]:
def oneHotEncoding(ds,rots):
    nds =[]
    nrots = novosRotulos(ds,rots)
    for i in range(len(ds)):
        l =[]
        for j in range(len(nrots)+1):
            l.append(0)
        nds.append(l)
    for i in range(len(ds)):
        for j in range(len(ds[0])-1):
            try:
                col = nrots.index(rots[j])
                #converter binarios
                if ds[i][j].lower() =="nao":
                    novoValor = 0
                elif ds[i][j].lower() =="sim":
                    novoValor = 1
                else:
                    novoValor = ds[i][j]  
                ###
                nds[i][col] = novoValor
            except(ValueError):
                col = nrots.index(ds[i][j])
                nds[i][col]=1
        nds[i][-1] = ds[i][-1]        
    return nrots,nds

In [30]:
rotulos = 'Formato','Fruta Cítrica','Cor', 'Rugosidade', 'Cheiro'
oneHotEncoding(dadosBrutos,rotulos)

(['esferico',
  'Fruta Cítrica',
  'vermelho',
  'alaranjado',
  'lisa',
  'rugosa',
  'Cheiro'],
 [[1, 1, 0, 1, 0, 1, 0, 'LARANJA'],
  [1, 0, 1, 0, 1, 0, 0, 'MACA'],
  [1, 1, 0, 1, 0, 1, 1, 'TANGERINA']])

In [31]:
dadosBrutos

[['esferico', 'sim', 'alaranjado', 'rugosa', 'nao', 'LARANJA'],
 ['esferico', 'nao', 'vermelho', 'lisa', 'nao', 'MACA'],
 ['esferico', 'sim', 'alaranjado', 'rugosa', 'sim', 'TANGERINA']]

In [32]:
nDs # codificada numericamente (label)

[[0, 0, 1, 1, 1, 'LARANJA'],
 [0, 1, 0, 0, 1, 'MACA'],
 [0, 0, 1, 1, 0, 'TANGERINA']]

In [33]:
#usando o sklearn
from sklearn.preprocessing import OneHotEncoder
X = [i[:-1] for i in nDs]
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X)

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)

In [34]:
enc.transform(X).toarray() # nem sempre fazem do jeito que queremos

array([[1., 1., 0., 0., 1., 0., 1., 0., 1.],
       [1., 0., 1., 1., 0., 1., 0., 0., 1.],
       [1., 1., 0., 0., 1., 0., 1., 1., 0.]])

In [35]:
dadosBrutos

[['esferico', 'sim', 'alaranjado', 'rugosa', 'nao', 'LARANJA'],
 ['esferico', 'nao', 'vermelho', 'lisa', 'nao', 'MACA'],
 ['esferico', 'sim', 'alaranjado', 'rugosa', 'sim', 'TANGERINA']]

In [36]:
rotulos

('Formato', 'Fruta Cítrica', 'Cor', 'Rugosidade', 'Cheiro')

In [37]:
#quem sabe o pandas
import pandas as pd
df = pd.DataFrame(data=[i[:-1] for i in dadosBrutos],columns=rotulos)

In [38]:
df.head()

,Formato,Fruta Cítrica,Cor,Rugosidade,Cheiro
0,esferico,sim,alaranjado,rugosa,nao
1,esferico,nao,vermelho,lisa,nao
2,esferico,sim,alaranjado,rugosa,sim


In [39]:
pd.get_dummies(df)

,Formato_esferico,Fruta Cítrica_nao,Fruta Cítrica_sim,Cor_alaranjado,Cor_vermelho,Rugosidade_lisa,Rugosidade_rugosa,Cheiro_nao,Cheiro_sim
0,1,0,1,1,0,0,1,1,0
1,1,1,0,0,1,1,0,1,0
2,1,0,1,1,0,0,1,0,1


<h3> Qual o problema com o one hot encoder??? </h3>

<h3> Dá pra ver que nossa solução é aparentemente melhor que as das bibliotecas </h3>

<h3> Um pouco mais </h3>  

In [40]:
# Um pouco mais
matrizAleatoria = np.random.rand(3,3)

In [41]:
matrizAleatoria

array([[0.63031433, 0.65970183, 0.37077206],
       [0.2471155 , 0.99843979, 0.98946601],
       [0.29694926, 0.5237234 , 0.70272943]])

In [42]:
def normaliza1(matriz,novomin,novomax):
    t = transposta(matriz)
    for i,l in enumerate(t):
        mi = min(l)
        mx = max(l)
        for j,c in enumerate(l):
            t[i][j] = (t[i][j] - mi) / (mx - mi) * (novomax - novomin) + novomin
    return transposta(t)

In [43]:
normaliza1(matrizAleatoria,0,1)

[[1.0, 0.2864414021429098, 0.0],
 [0.0, 1.0, 1.0],
 [0.1300467406828484, 0.0, 0.5365453617501951]]

In [44]:
# ou usando mais o numpy

In [45]:
def normaliza2(matriz,novomin,novomax):
    t = matriz.T
    for i,l in enumerate(t):
        mi = min(l)
        mx = max(l)
        for j,c in enumerate(l):
            t[i][j] = (t[i][j] - mi) / (mx - mi) * (novomax - novomin) + novomin
    return t.T

In [46]:
normaliza2(matrizAleatoria,0,1)

array([[1.        , 0.2864414 , 0.        ],
       [0.        , 1.        , 1.        ],
       [0.13004674, 0.        , 0.53654536]])

In [47]:
# ou ainda
def normaliza3(t,novomin,novomax):
    mi = np.amin(t,axis=0)
    mx = np.amax(t,axis=0)
    for i,l in enumerate(t):
        for j,c in enumerate(l):
            t[i][j] = (t[i][j] - mi[j]) / (mx[j] - mi[j]) * (novomax - novomin) + novomin
    return t

In [48]:
normaliza3(matrizAleatoria,0,1)

array([[1.        , 0.2864414 , 0.        ],
       [0.        , 1.        , 1.        ],
       [0.13004674, 0.        , 0.53654536]])

In [49]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(matrizAleatoria)
MinMaxScaler(copy=True, feature_range=(0, 1))
print(scaler.transform(matrizAleatoria))

[[1.         0.2864414  0.        ]
 [0.         1.         1.        ]
 [0.13004674 0.         0.53654536]]


<h1> Enjoy </h1>